In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [6]:
X_train_mat = pd.read_csv('data/X_train_mat.csv', index_col=0)
Y_train = pd.read_csv('data/Y_train.csv', index_col=0)
X_val_mat = pd.read_csv('data/X_val_mat.csv', index_col=0)
Y_val = pd.read_csv('data/Y_val.csv', index_col=0)

X_test = pd.concat([pd.read_csv('data/Xte{}.csv'.format(i), index_col=0) for i in range(3)])

X_test_mat = pd.concat([
    pd.read_csv('data/Xte{}_mat100.csv'.format(i), delimiter=' ', header=None) 
    for i in range(3)
])



In [7]:
Xtr = X_train_mat.values
# Y to {-1, 1}
Ytr = Y_train.values * 2 - 1

Xval = X_val_mat.values
# Y to {-1, 1}
Yval = Y_val.values * 2 - 1

In [8]:
from Ridge import RidgeRegression

In [9]:
from random import sample

def create_k_folds(X,Y,k=10):
    train_test_split = []
    size = len(X)
    num_of_elements = int(size / k)
    dataset=list(zip(X, Y))
    for i in range(k):
        index_value =sample(list(enumerate(dataset)), num_of_elements)
        indexes=[]
        new_sample=[]
        for idx, val in index_value:
            indexes.append(idx)
            new_sample.append(val)
        train_test_split.append(new_sample)
        for idx in sorted(indexes, reverse=True):
            dataset.pop(idx)
    if len(dataset) != 0:
        for rows in range(len(dataset)):
            train_test_split[rows].append(dataset[rows])
        dataset.clear()
    return train_test_split

In [10]:
train_test_split=create_k_folds(Xtr,Ytr)

In [12]:
from itertools import chain

def kfolds_cv(train_test_split): 
    mean_errors=[]
    lambdas=[0.001,0.02,0.1,0.3,1,5,10]
    for j,lambda_ in enumerate(lambdas):
        X=[[] for i in range(10)]
        Y=[[] for i in range(10)]
        error=[[] for i in range(7)]
        for i in range(10):
            X[i],Y[i]=zip(*train_test_split[i])
        for i in range(10):
            Xtrain= X[:i] + X[i+1:]
            Ytrain= Y[:i]+ Y[i+1:]
            Xtrain=list(chain.from_iterable(Xtrain))
            Ytrain=list(chain.from_iterable(Ytrain))
            Xvalidation=X[i]
            Yvalidation=Y[i]
            ridge = RidgeRegression(Xtrain,Ytrain,kernel='rbf',reg=lambda_)
            ridge.fit()
            error[j].append(ridge.compute_misclassif_error(Xvalidation, Yvalidation))
        mean_errors.append(np.mean(np.array(error[j])))
    lowest_error_idx=np.argmin(np.array(mean_errors))
    best_lambda=lambdas[lowest_error_idx]
    return best_lambda
    
    

In [ ]:
lambda_=kfolds_cv(train_test_split)